In [14]:
import numpy as np
import time
import random
from numba import njit
from IPython.display import clear_output

In [15]:
@njit()
def _calculate_cost(chromosome, flow_matrix, distance_matrix):
  cost = 0
  n = chromosome.shape[0]
  for i in range(n):
    for j in range(n):
      cost += distance_matrix[i][j]*flow_matrix[chromosome[i]][chromosome[j]]
  return cost

@njit()
def _insertValues(child, parent, n, crossover_point):
    while(child.shape[0] != n):
      if(parent[crossover_point] not in child):
        child = np.append(child, parent[crossover_point])
      crossover_point = (crossover_point + 1) % n
    return child

@njit()
def _cutAndCrossfill(father,mother,n):
  
  crossover_point = random.randint(0, n)
  child1 = np.empty(n)
  child2 = np.empty(n)
  
  child1 = father[:crossover_point]
  child2 = mother[:crossover_point]
  child1 = _insertValues(child1,mother,n,crossover_point)
  child2 = _insertValues(child2,father,n,crossover_point)

  return child1,child2

In [16]:
class Individual:

  def __init__(self, size,chromosome):
    self.chromosome = chromosome
    self.size = size
    self.cost = np.inf

  def __str__(self):
    return str(self.chromosome)

  def __lt__(self,other):
    return self.cost < other.cost

  def __gt__(self,other):
    return self.cost > other.cost

  def __eq__(self,other):
    return self.cost == other.cost
  
  @staticmethod
  def generateRandom(n):
    return Individual(n,np.random.permutation(n))

  def mutation(self, mut_prob):
    if random.random() <= mut_prob:
      mp1 = random.randint(0, self.size - 1)
      mp2 = random.randint(0, self.size - 1)
      self.chromosome[mp1], self.chromosome[mp2] = self.chromosome[mp2], self.chromosome[mp1]

  def cutAndCrossfill(self,parent,cro_prob):
    
    if(random.random() > cro_prob):
      return Individual(self.size,self.chromosome), Individual(parent.size,parent.chromosome)
    
    n = self.size

    child1 = Individual(n,[])
    child2 = Individual(n,[])
    child1.chromosome,child2.chromosome = _cutAndCrossfill(self.chromosome,parent.chromosome,n)

    return (child1, child2)

  def calculate_cost(self,flow_matrix,distance_matrix):  
    self.cost = _calculate_cost(self.chromosome, flow_matrix,distance_matrix)


In [17]:
def bestTwoOutOfFive(population):
    # Select 5 individuals randomly
    tournament = [population[random.randint(
        0, len(population)-1)] for _ in range(len(population)//5)]

    # return best 2 of five
    return sorted(tournament)[:2]

In [18]:
def geneticAgorithm(f, d, pop_size, cro_prob, mut_prob, gen_size, off_size):
    """

    Genetic Algorithm for Quadratic Assignment Problem
    __________________________________________________
    
    f: Flow matrix (Numpy array of n x n dimentions)
    d: Distance matrix (Numpy array of n x n dimentions)
    pop_size: Population size (Integer)
    cro_prob: Crossover probability (Float [0.0,1.0))
    mut_prob: Mutation proability (Float [0.0,1.0))
    gen_size: Number of generations (Integer)
    off_size: Offspring size (Integer)
    """
    # Variables Initialization
    
    generations = 1
    n = len(f[0])
    offspring = []

    # INITIALISE population with random candidate solutions
    population = [Individual.generateRandom(n) for _ in range(pop_size)]

    best = min(population)
    # EVALUATE each candidate    
    for ind in population:
        ind.calculate_cost(f,d)

    # Repeat until termination condition is satisfied
    while(generations < gen_size + 1):

        for i in range(off_size//2):
            # PARENT SELECTION (best two out of five)
            father, mother = bestTwoOutOfFive(population)

            # CROSSOVER
            child1, child2 = father.cutAndCrossfill(mother, cro_prob)

            # MUTATION
            child1.mutation(mut_prob)
            child2.mutation(mut_prob)

            # EVALUATION OF NEW CANDIDATES
            child1.calculate_cost(f,d)
            child2.calculate_cost(f,d)

            offspring.append(child1)
            offspring.append(child2)
            
        # SURVIVOR SELECTION
        population = sorted(offspring)[:pop_size]
        offspring = []

        generations += 1

    return population


In [19]:
def read_qap_dataset(filename):
    def read_integers(filename):
        with open(filename) as f:
            return [int(elem) for elem in f.read().split()]

    file_it = iter(read_integers(filename))
    # Number of points
    n = next(file_it)
    # Distance between locations
    A = [[next(file_it) for j in range(n)] for i in range(n)]
    # Flow between factories
    B = [[next(file_it) for j in range(n)] for i in range(n)]
    
    return (np.array(B), np.array(A))


In [20]:
def test(pop_size, cro_prob, mut_prob, gen_size, off_size, runs, filename):
    runs_stats = []

    f, d = read_qap_dataset(filename)

    for i in range(runs):
        print('Run number: {}'.format(i))
        begining = time.time()
        p = geneticAgorithm(f,d,pop_size,cro_prob,mut_prob,gen_size,off_size)
        end = time.time()

        total_time = end - begining
        print(f'Total time: {total_time}')
        best = p[0]
        worst = p[-1]
        average_cost = np.mean([ind.cost for ind in p])
        runs_stats.append({ 'runtime': total_time, 'best': best, 'worst': worst, 'average_cost': average_cost })
    
    costs = [ stat['average_cost'] for stat in runs_stats ]
    times = [ stat['runtime'] for stat in runs_stats ]
 
    best_individual = min([ stat['best'] for stat in runs_stats ])
    worst_individual = max([ stat['worst'] for stat in runs_stats ])

    cost_average = np.mean(costs)
    cost_std = np.std(costs)
    
    time_average = np.mean(times)
    time_std = np.std(times)
    

    max_time = max(times)
    min_time = min(times)

    print(f'Costo promedio: {cost_average}')
    print(f'Desviación estandar de costos: {cost_std}')
    print(f'Mejor solución encontrada: {best_individual}: {best_individual.cost}')
    print(f'Peor solución encontrada: {worst_individual}: {worst_individual.cost}')
    print(f'Tiempo promedio de ejecución: {time_average}')
    print(f'Desviación estandar tiempo de ejecución: {time_std}')
    print(f'Valor máximo (tiempo de ejecución): {max_time}')
    print(f'Valor mínimo (tiempo de ejecución): {min_time}')


## Parámetros 1
* Tamaño de población: 300
* Número de descendendientes: 600
* Número máximo de generaciones: 50,000
* Probabilidad de cruzamiento: 0.9
* Probabilidad de mutación: 0.2
* Número de corridas: 1
* Dataset: tho40

In [ ]:
POP_SIZE = 300
CRO_PROB = 0.9
MUT_PROB = 0.2
GEN_SIZE = 50000
OFF_SIZE = 600
RUNS = 1
FILENAME = 'tho40.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

## Parámetros 2
* Tamaño de población: 300
* Número de descendendientes: 600
* Número máximo de generaciones: 10,000
* Probabilidad de cruzamiento: 0.9
* Probabilidad de mutación: 0.2
* Número de corridas: 1
* Dataset: tho40

In [24]:
POP_SIZE = 300
CRO_PROB = 0.9
MUT_PROB = 0.2
GEN_SIZE = 10000
OFF_SIZE = 600
RUNS = 1
FILENAME = 'tho40.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

10000
Total time: 172.0122230052948
Costo promedio: 248970.0
Desviación estandar de costos: 0.0
Mejor solución encontrada: [37 30 26 25 20 23 39  2 22  4 12 15 31  9  7 16 38 19  0 14 13 10  8 33
  3 36 21 32  5 29 11 24 18  1 17 27 34 28  6 35]: 248970
Peor solución encontrada: [37 30 26 25 20 23 39  2 22  4 12 15 31  9  7 16 38 19  0 14 13 10  8 33
  3 36 21 32  5 29 11 24 18  1 17 27 34 28  6 35]: 248970
Tiempo promedio de ejecución: 172.0122230052948
Desviación estandar tiempo de ejecución: 0.0
Valor máximo (tiempo de ejecución): 172.0122230052948
Valor mínimo (tiempo de ejecución): 172.0122230052948


## Parámetros 3
* Tamaño de población: 100 ✅
* Número de descendendientes: 200 ✅
* Número máximo de generaciones: 10,000 ✅
* Probabilidad de cruzamiento: 0.9
* Probabilidad de mutación: 0.2
* Número de corridas: 30
* Dataset: tho40

In [ ]:
POP_SIZE = 100
CRO_PROB = 0.9
MUT_PROB = 0.2
GEN_SIZE = 10000
OFF_SIZE = 200
RUNS = 30
FILENAME = 'tho40.dat'
test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

## Parámetros 4
* Tamaño de población: 60 ✅
* Número de descendendientes: 120 ✅
* Número máximo de generaciones: 10,000 ✅
* Probabilidad de cruzamiento: 0.9
* Probabilidad de mutación: 0.2
* Número de corridas: 30
* Dataset: tho40

In [21]:
POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.2
GEN_SIZE = 10000
OFF_SIZE = 120
RUNS = 30
FILENAME = 'tho40.dat'
test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

Run number: 0
Total time: 13.107828378677368
Run number: 1
Total time: 12.735949516296387
Run number: 2


KeyboardInterrupt: 

In [22]:
POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.5
GEN_SIZE = 10000
OFF_SIZE = 120
RUNS = 30
FILENAME = 'tho40.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

Run number: 0
Total time: 13.580076217651367
Run number: 1
Total time: 13.566738605499268
Run number: 2
Total time: 13.595036268234253
Run number: 3
Total time: 13.600423336029053
Run number: 4
Total time: 13.584847688674927
Run number: 5
Total time: 13.525022745132446
Run number: 6
Total time: 13.569252967834473
Run number: 7
Total time: 13.518487930297852
Run number: 8
Total time: 13.562052965164185
Run number: 9
Total time: 13.520960807800293
Run number: 10
Total time: 13.574140548706055
Run number: 11
Total time: 13.558037281036377
Run number: 12
Total time: 13.556086540222168
Run number: 13
Total time: 13.574465990066528
Run number: 14
Total time: 13.54203724861145
Run number: 15
Total time: 13.561954975128174
Run number: 16
Total time: 13.511204481124878
Run number: 17
Total time: 13.492273330688477
Run number: 18
Total time: 13.510138511657715
Run number: 19
Total time: 13.441222429275513
Run number: 20
Total time: 13.496401071548462
Run number: 21
Total time: 13.447210550308228

In [23]:
POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.8
GEN_SIZE = 10000
OFF_SIZE = 120
RUNS = 30
FILENAME = 'tho40.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

Run number: 0
Total time: 14.58254337310791
Run number: 1
Total time: 14.596424579620361
Run number: 2
Total time: 14.615927457809448
Run number: 3
Total time: 14.54718017578125
Run number: 4
Total time: 14.543163299560547
Run number: 5
Total time: 14.56619906425476
Run number: 6
Total time: 14.540175199508667
Run number: 7
Total time: 14.48717451095581
Run number: 8
Total time: 14.5388343334198
Run number: 9
Total time: 14.491521120071411
Run number: 10
Total time: 14.506419658660889
Run number: 11
Total time: 14.487808465957642
Run number: 12
Total time: 14.545653820037842
Run number: 13
Total time: 14.549883842468262
Run number: 14
Total time: 14.52515459060669
Run number: 15
Total time: 14.547105312347412
Run number: 16
Total time: 14.553207874298096
Run number: 17
Total time: 14.493157625198364
Run number: 18
Total time: 14.497419834136963
Run number: 19
Total time: 14.516160249710083
Run number: 20
Total time: 14.490182638168335
Run number: 21
Total time: 14.555657148361206
Run n

In [24]:

POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.8
GEN_SIZE = 10000
OFF_SIZE = 120
RUNS = 30
FILENAME = 'lipa20a.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

Run number: 0
Total time: 10.824085235595703
Run number: 1
Total time: 10.761377573013306
Run number: 2
Total time: 10.751070737838745
Run number: 3
Total time: 10.681461334228516
Run number: 4
Total time: 10.744446039199829
Run number: 5
Total time: 10.747729063034058
Run number: 6
Total time: 10.704816102981567
Run number: 7
Total time: 10.79611587524414
Run number: 8
Total time: 10.83516001701355
Run number: 9
Total time: 10.765346765518188
Run number: 10
Total time: 10.78132176399231
Run number: 11
Total time: 10.678722858428955
Run number: 12
Total time: 10.74444031715393
Run number: 13
Total time: 10.6721031665802
Run number: 14
Total time: 10.774620532989502
Run number: 15
Total time: 10.731037855148315
Run number: 16
Total time: 10.814932346343994
Run number: 17
Total time: 10.74869990348816
Run number: 18
Total time: 10.806140899658203
Run number: 19
Total time: 10.741754055023193
Run number: 20
Total time: 10.76575255393982
Run number: 21
Total time: 10.728252649307251
Run nu

In [25]:
POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.8
GEN_SIZE = 10000
OFF_SIZE = 120
RUNS = 30
FILENAME = 'kra30a.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

Run number: 0
Total time: 12.61998963356018
Run number: 1
Total time: 12.587009191513062
Run number: 2
Total time: 12.58946418762207
Run number: 3
Total time: 12.635534763336182
Run number: 4
Total time: 12.509979724884033
Run number: 5
Total time: 12.644089698791504
Run number: 6
Total time: 12.590309381484985
Run number: 7
Total time: 12.61310863494873
Run number: 8
Total time: 12.648937463760376
Run number: 9
Total time: 12.526271343231201
Run number: 10
Total time: 12.589993238449097
Run number: 11
Total time: 12.54756498336792
Run number: 12
Total time: 12.648331642150879
Run number: 13
Total time: 12.51822566986084
Run number: 14
Total time: 12.657963514328003
Run number: 15
Total time: 12.536811351776123
Run number: 16
Total time: 12.63651728630066
Run number: 17
Total time: 12.577388286590576
Run number: 18
Total time: 12.681981801986694
Run number: 19
Total time: 12.558942794799805
Run number: 20
Total time: 12.560433149337769
Run number: 21
Total time: 12.476117372512817
Run 

In [26]:
POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.8
GEN_SIZE = 10000
OFF_SIZE = 120
RUNS = 30
FILENAME = 'tai35b.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

Run number: 0
Total time: 13.306292295455933
Run number: 1
Total time: 13.305376529693604
Run number: 2
Total time: 13.398233413696289
Run number: 3
Total time: 13.396032094955444
Run number: 4
Total time: 13.354853868484497
Run number: 5
Total time: 13.398320436477661
Run number: 6
Total time: 13.301538944244385
Run number: 7
Total time: 13.327587366104126
Run number: 8
Total time: 13.344403505325317
Run number: 9
Total time: 13.34552788734436
Run number: 10
Total time: 13.35220742225647
Run number: 11
Total time: 13.377114295959473
Run number: 12
Total time: 13.376908302307129
Run number: 13
Total time: 13.368209838867188
Run number: 14
Total time: 13.335145473480225
Run number: 15
Total time: 13.361910343170166
Run number: 16
Total time: 13.338172674179077
Run number: 17
Total time: 13.372073411941528
Run number: 18
Total time: 13.350721836090088
Run number: 19
Total time: 13.368232011795044
Run number: 20
Total time: 13.35120153427124
Run number: 21
Total time: 13.271435976028442
R

**Recursos**

**Test cases**
https://neos-guide.org/content/quadratic-assignment-problem#:~:text=The%20quadratic%20assignment%20problem%20(QAP)%20was%20introduced%20by%20Koopmans%20and,minimize%20the%20total%20assignment%20cost

**Datasets**
http://www.mgi.polymtl.ca/anjos/qaplib/inst.html

**Todo**

**Prioridad Alta**
*   Esqueleto Clase (Dani) ✅✅
*   Esqueleto Algoritmo (Morfin) ✅✅
*   Mutación (Dani) ✅✅
*   Cruzamiento (Morfin) ✅✅
*   Selección de Padres (Dani) ✅✅
*   Reemplazo (Morfin) ✅✅
*   Fitness (Dani) ✅✅

**Prioridad Media**
*   Leer el archivo (Ambos) ✅✅
*   Calcular STD, AVG, max, min (Dani)✅✅
*   Guardar en archivo la mejor solución (Dani)

**Prioridad Baja**
*   Calibrar parámetros

**Necesario**
*   Redactar el reporte